<a href="https://colab.research.google.com/github/alayacruz/CynapticsInductions/blob/main/aivsreal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from tensorflow.keras.utils import load_img
from keras.models import Sequential
from keras.applications import MobileNetV2, ResNet152, VGG16, EfficientNetB0, InceptionV3
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from sklearn.preprocessing import LabelEncoder
from tqdm.notebook import tqdm
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:

from PIL import Image
from IPython.display import display

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def createdataframe(dir):
    image_paths=[]
    labels=[]
    for label in os.listdir(dir):
#os.listdir(dir) returns a list containing the names of the entries in the directory given by dir.
      for imagename in os.listdir(os.path.join(dir,label)):
         image_path = os.path.join(dir, label, imagename)
         if os.path.isfile(image_path):
          image_paths.append(os.path.join(dir,label,imagename)) #OR tf.keras.utils.image_dataset_from_directory('directory_name')
          labels.append(label)
      print(label,"completed")
    return image_paths,labels

In [ ]:
def extract_features(images):
  features=[]
  valid_image_paths=[]
  for image_path in tqdm(images):
    img=Image.open(image_path)
    if img.format not in ['JPEG', 'PNG', 'GIF', 'BMP', 'JPG', 'jpg']:  # Add other supported formats if needed
        print(f"Warning: Unsupported image format '{img.format}' for image: {image_path}")
        continue
    img=load_img(image_path,target_size=(128,128))
    img=np.array(img)
    features.append(img)


  features=np.array(features)
  features=features.reshape(features.shape[0],128,128,3)
  return features


In [ ]:
TRAIN_DIR="/content/drive/MyDrive/train/Train"
train=pd.DataFrame()
train['image'],train['label']=createdataframe(TRAIN_DIR)
train_features=extract_features(train['image'])
x_train=train_features/255.0 #to scale pixel values(ranging from 0 to 255) from 0 to 1
os.listdir(TRAIN_DIR)

AI completed
Real completed


  0%|          | 0/811 [00:00<?, ?it/s]

['AI', 'Real']

In [ ]:
le=LabelEncoder()
le.fit(train['label'])
y_train=le.transform(train['label'])
y_train=to_categorical(y_train,num_classes=2) # to_categorical by keras converts class vector into binary class matrix
print(le.classes_)


['AI' 'Real']


In [ ]:
early_stopping= EarlyStopping(
    monitor='accuracy',
    mode='max',
    patience=4,
    restore_best_weights=True
)

In [ ]:
model=Sequential()
model.add(Conv2D(32,(3,3),1,activation='relu',input_shape=(128,128,3)))
model.add(MaxPooling2D())

model.add(Conv2D(256, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(512, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
model.add(Dense(2,activation='sigmoid')) #sigmoid instead of softmax

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()
"""if x_train.shape[0] == 0 or y_train.shape[0] == 0:
    raise ValueError("Training data is empty. Please check your data loading process.")"""

batch_size=16
if batch_size > x_train.shape[0]:
    batch_size = 8 # Set batch size to the number of samples if it's larger
    print(f"Warning: Batch size is larger than the number of training samples. ")
model.fit(x=x_train,y=y_train,batch_size=batch_size,epochs=10,callbacks=[early_stopping])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 256)         │          73,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 100352)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │      51,380,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 52,800,258 (201.42 MB)

 Trainable params: 52,800,258 (201.42 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 13s 133ms/step - accuracy: 0.5939 - loss: 0.7892
Epoch 2/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 11s 39ms/step - accuracy: 0.9004 - loss: 0.2698
Epoch 3/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.9202 - loss: 0.2336
Epoch 4/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.9493 - loss: 0.1257
Epoch 5/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9407 - loss: 0.2757
Epoch 6/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step - accuracy: 0.9776 - loss: 0.0795
Epoch 7/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.9739 - loss: 0.0812
Epoch 8/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.9901 - loss: 0.0223
Epoch 9/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.9905 - loss: 0.0417
Epoch 10/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.9902 - loss: 0.1290


In [ ]:
test_image_dir='/content/drive/MyDrive/test_again2/Test'
import imghdr

def create_test_dataframe(dir):
    image_paths = []
    image_names = []
    for filename in os.listdir(dir):
      file_path = os.path.join(dir, filename)
      if imghdr.what(file_path) is not None:
        #if filename.endswith('.jpg'):
         image_paths.append(file_path)
         image_names.append(filename)

    return image_paths,image_names


In [ ]:
label_mapping = {0: 'AI', 1: 'Real'}

In [ ]:
test=pd.DataFrame()
test['image'],test['image_name']=create_test_dataframe(test_image_dir)
print(test['image_name'])
test_features=extract_features(test['image'])
x_test=test_features/255.0

0      image_115.jpg
1      image_103.jpg
2      image_112.jpg
3      image_114.jpg
4      image_106.jpg
           ...      
194     image_62.jpg
195     image_26.jpg
196     image_73.jpg
197     image_33.jpg
198    image_183.jpg
Name: image_name, Length: 199, dtype: object


  0%|          | 0/199 [00:00<?, ?it/s]

In [ ]:
predictions=model.predict(x_test)
predicted_labels=np.argmax(predictions,axis=1)

results=pd.DataFrame({'Id': test['image_name'], 'label':predicted_labels})
results['label']=results['label'].map(label_mapping)
results.to_csv('predictions3.csv',header=True, index=False)

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 160ms/step
